# 4章 ニューラルネットワークの学習

学習を行えるようにするため `損失関数` という指標を導入する  

> コンピュータビジョンの分野で有名な特徴 量としては、SIFT や SURF、HOG などが挙げられます



In [5]:
import numpy as np

# 最小二乗誤差
def mean_squared_error(y, t):
    return 0.5 * np.sum((y - t) ** 2)

In [2]:
# 2 の確率が一番高い場合の損失関数の値
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
mean_squared_error(np.array(y), np.array(t))

0.097500000000000031

In [4]:
# 7 の確率が一番高い場合の損失関数の値
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.5, 0.0]
mean_squared_error(np.array(y), np.array(t))

0.72250000000000003

In [6]:
import numpy as np

# 交差エントロピー誤差

def cross_entropy_error(y, t):
    delta = 1e-7
    return - np.sum(t * np.log(y + delta))

In [7]:
# 2 の確率が一番高い場合の損失関数の値
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

0.51082545709933802

In [8]:
# 7 の確率が一番高い場合の損失関数の値
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.5, 0.0]
cross_entropy_error(np.array(y), np.array(t))

2.3025840929945458

### 次回、4.2.3 ミニバッチ学習

デュエルスタンバイ